In [ ]:
from model import MiniCharGPTLM
from tokenizer import CharTokenizer
import torch

model_path = "model.pth"
ckpt = torch.load(model_path)

tokenizer = CharTokenizer()
model = MiniCharGPTLM(seqlen=ckpt["seqlen"],
                      h_dim=ckpt["h_dim"],
                      ff_dim=ckpt["ff_dim"],
                      n_head=ckpt["n_head"],
                      n_block=ckpt["n_block"],
                      n_token=len(tokenizer.char2id))

model.load_state_dict(ckpt["state_dict"])

In [ ]:
import random

length = 32
k = 2
text = "a"
for l in range(length):
    inputs = tokenizer(text, truncate=True, padding=True, max_length=ckpt["seqlen"])
    inputs = torch.tensor(inputs)
    inputs.unsqueeze(0)

    model.eval()
    with torch.no_grad():
        out = model.forward(inputs)
    
    # top k sampling
    out = torch.topk(out.cpu(), k).indices[0]
    out = random.choice(out).item()

    # greedy searching using k=1 or this
    # out = out.argmax(-1).cpu().item()
    next_char = tokenizer.id2char[out]
    if next_char == tokenizer.end_token:
        break
    text += next_char

In [ ]:
print(text)